<h1>Содержание.<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Задача-проекта" data-toc-modified-id="Задача-проекта-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Задача проекта</a></span></li><li><span><a href="#Подготока-данных" data-toc-modified-id="Подготока-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Подготока данных</a></span><ul class="toc-item"><li><span><a href="#Функции-обработки" data-toc-modified-id="Функции-обработки-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Функции обработки</a></span></li><li><span><a href="#Анализ-данных." data-toc-modified-id="Анализ-данных.-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Анализ данных.</a></span></li><li><span><a href="#Отсеивание-стран" data-toc-modified-id="Отсеивание-стран-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Отсеивание стран</a></span></li></ul></li><li><span><a href="#Обучение-моделей" data-toc-modified-id="Обучение-моделей-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение моделей</a></span><ul class="toc-item"><li><span><a href="#Подготовка-данных." data-toc-modified-id="Подготовка-данных.-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Подготовка данных.</a></span></li><li><span><a href="#Обучение-моделей" data-toc-modified-id="Обучение-моделей-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Обучение моделей</a></span></li></ul></li><li><span><a href="#Проверка-модели" data-toc-modified-id="Проверка-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка модели</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li></ul></div>

## Задача проекта

**Цель:**
Разработка ML модели для подбора наиболее подходящих названий с geonames.

**Задачи:**
- Создать решение для подбора наиболее подходящих названий с geonames. Например Ереван -> Yerevan
- На примере РФ и стран наиболее популярных для релокации - Беларусь, Армения, Казахстан, Кыргызстан, Турция, Сербия. Города с населением от 15000 человек (с возможностью масштабирования на сервере заказчика)
- Возвращаемые поля geonameid, name, region, country, cosine similarity
- формат данных на выходе: список словарей, например [{dict_1}, {dict_2}, …. {dict_n}] где словарь - одна запись с указанными полями

## Подготока данных

Импортируем библиотеки.

In [1]:
import diffusers
import numpy as np
import pandas as pd
import random
import safetensors
import warnings

from IPython.display import Markdown, display
from matplotlib import pyplot as plt
import plotly.express as px
from sentence_transformers import SentenceTransformer, util

from sqlalchemy import create_engine, inspect
from sqlalchemy.engine.url import URL

warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

Установка некоторых параметров.

In [2]:
# PostgreSQL
DATABASE = {
    'drivername': 'postgresql',
    'username': 'postgres',
    'password': 'postgres',
    'host': 'localhost',
    'port': 5432,
    'database': 'postgres',
    'query': {}
}

# Переменная векторного кодирования
EMBEDDER = None

# Значение
RANDOM_STATE = 12345

# Коды выбранных стран
COUNTRY_CODES = ['RU', 'BY', 'KG', 'KZ', 'AM', 'TR', 'RS']

### Функции обработки

Возвращение похожих записей к запросу

In [3]:
def similar(query, top_k=1, is_dictionary=False):
    query_embedding = EMBEDDER.encode(query, convert_to_tensor=False)
    hits = util.semantic_search(query_embedding,None, top_k=top_k)
    hits = hits[0]
    result_rows = []
    for hit in hits:
        selected_columns = ['name_city','concatenated_codes','name_admin','country']
        result_row = selected_cities.loc[hit['corpus_id'], selected_columns].to_frame().T
        result_row['Score'] = hit['score']
        result_rows.append(result_row)

    result_df = pd.concat(result_rows, ignore_index=True)
    new_column_names = ['name','code','region','country','similarity']
    result_df.columns = new_column_names

    if is_dictionary:
        return result_df.to_dict(orient='records')
    else:
        return result_df

Оценка точности.

In [4]:
def accuracy_score(y_true, y_pred):
    if y_true.shape != y_pred.shape:
        raise ValueError('Ошибка размерности')
    
    matches = []
    for index, (true_row, pred_row) in enumerate(
        zip(y_true.itertuples(index=False), y_pred.itertuples(index=False))):
            matches.append(1 if true_row == pred_row else 0)
    accuracy = sum(matches) / len(matches) #средняя точность
    return accuracy

Функция предсказания.

In [5]:
def predict_val(queries):
    predictions = pd.concat(
        queries.apply(similar(query, top_k=1)[['name', 'region', 'country']]).tolist(),ignore_index=True)
    return predictions

### Анализ данных.

Для работы будут использоваться данные с сайта `Geonames.org`

Выборка будет состоять из следующих файлов:
- admin1CodesASCII - административные коды городов
- cities15000 - названия городов и их данные
- countryInfo - коды стран

Выгрузим данные и посмотрим их основные характеристики.

**CountryInfo**

In [6]:
countries = pd.read_csv('countryInfo.txt', delimiter='\t', header = None, index_col=None, dtype={'geonameid': 'Int64'},
              names=['country_code','iso3','iso-numeric','fips','country','capital','area(in_sq_km)','population',
                   'continent','tld','currency_code','currency_name','phone','postal_code_format','postal_code_regex',
                   'languages','geonameid','neighbours','equivalent_fips_code'],  
              usecols=['country_code','country'] ).dropna().drop_duplicates() 
countries.reset_index(drop=True, inplace=True) 

In [7]:
print(countries.info())
print(countries.describe())
countries.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   country_code  252 non-null    object
 1   country       252 non-null    object
dtypes: object(2)
memory usage: 4.1+ KB
None
       country_code  country
count           252      252
unique          252      252
top            #ISO  Country
freq              1        1


,country_code,country
0,#ISO,Country
1,AD,Andorra
2,AE,United Arab Emirates
3,AF,Afghanistan
4,AG,Antigua and Barbuda
5,AI,Anguilla
6,AL,Albania
7,AM,Armenia
8,AO,Angola
9,AQ,Antarctica


**Cities15000**

In [8]:
cities15000 = pd.read_csv('cities15000.txt', delimiter='\t', header = None, index_col=None, 
                names=['geonameid','name','asciiname','alternate_names','latitude','longitude','feature_class',
                       'feature_code','country_code','cc2','admin1_code','admin2_code','admin3_code','admin4_code',
                       'population','elevation','dem','timezone','modification_date'],
                usecols=['geonameid','country_code','name','admin1_code','population']).dropna().drop_duplicates() 
cities15000.reset_index(drop=True, inplace=True)

In [9]:
print(cities15000.info())
print(cities15000.describe())
cities15000.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27156 entries, 0 to 27155
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   geonameid     27156 non-null  int64 
 1   name          27156 non-null  object
 2   country_code  27156 non-null  object
 3   admin1_code   27156 non-null  object
 4   population    27156 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 1.0+ MB
None
          geonameid    population
count  2.715600e+04  2.715600e+04
mean   2.854721e+06  1.223810e+05
std    2.208807e+06  5.293418e+05
min    1.057000e+04  0.000000e+00
25%    1.272944e+06  2.205725e+04
50%    2.514236e+06  3.565750e+04
75%    3.576220e+06  7.445750e+04
max    1.268714e+07  2.231547e+07


,geonameid,name,country_code,admin1_code,population
0,3040051,les Escaldes,AD,08,15853
1,3041563,Andorra la Vella,AD,07,20430
2,290594,Umm Al Quwain City,AE,07,62747
3,291074,Ras Al Khaimah City,AE,05,351943
4,291580,Zayed City,AE,01,63482
5,291696,Khawr Fakkān,AE,06,40677
6,292223,Dubai,AE,03,3478300
7,292231,Dibba Al-Fujairah,AE,04,30000
8,292239,Dibba Al-Hisn,AE,04,26395
9,292672,Sharjah,AE,06,1274749


**Admin1CodesASCII**

In [10]:
admin_codes = pd.read_csv('admin1CodesASCII.txt', delimiter='\t', header = None, index_col=None, 
                names=['concatenated_codes','name','ascii_name','geonameid'], 
                usecols=['concatenated_codes','name']).dropna().drop_duplicates() 
admin_codes.reset_index(drop=True, inplace=True) 

In [11]:
print(admin_codes.info())
print(admin_codes.describe())
admin_codes.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3881 entries, 0 to 3880
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   concatenated_codes  3881 non-null   object
 1   name                3881 non-null   object
dtypes: object(2)
memory usage: 60.8+ KB
None
       concatenated_codes          name
count                3881          3881
unique               3881          3782
top                 AD.06  Saint George
freq                    1             6


,concatenated_codes,name
0,AD.06,Sant Julià de Loria
1,AD.05,Ordino
2,AD.04,La Massana
3,AD.03,Encamp
4,AD.02,Canillo
5,AD.07,Andorra la Vella
6,AD.08,Escaldes-Engordany
7,AE.07,Imārat Umm al Qaywayn
8,AE.05,Raʼs al Khaymah
9,AE.03,Dubai


Пропущенных данных ни в одной таблице нет --> можно спокойно дальше работать с данными.

В таблице admin_codes столбец с кодами содержит как код страны, так и административный код, поэтому разделим это столбец на два разных.

In [12]:
admin_codes[['country_code', 'admin1_code']] = admin_codes['concatenated_codes'].str.split('.', expand=True)

Для дальнейшей работы нам понадобится датафрейм, включающий в себя объединенные данные из всез 3х таблиц.

In [13]:
joined_df = pd.merge(
    pd.merge(admin_codes,countries, on='country_code', how='left'), cities15000, on=['country_code', 'admin1_code'], 
    how='left', suffixes=('_admin', '_city')).dropna() 

Выведем случайную выборку из получившихся данных.

In [14]:
joined_df.sample(10, random_state=RANDOM_STATE)

,concatenated_codes,name_admin,country_code,admin1_code,country,geonameid,name_city,population
23984,UA.17,Odessa,UA,17,Ukraine,712886.0,Balta,18511.0
12893,IN.23,Punjab,IN,23,India,1262596.0,Mukeriān,22751.0
26113,US.NY,New York,US,NY,United States,5134453.0,Rotterdam,20652.0
14418,IQ.02,Basra,IQ,02,Iraq,89824.0,Umm Qaşr,107620.0
12012,IN.28,West Bengal,IN,28,India,1271670.0,Gangārāmpur,65316.0
24346,US.FL,Florida,US,FL,United States,4160705.0,Kendale Lakes,56148.0
9659,FR.32,Hauts-de-France,FR,32,France,2995908.0,Marcq-en-Barœul,38629.0
2660,BR.05,Bahia,BR,05,Brazil,3456696.0,Morro do Chapéu,21670.0
11231,HT.11,Ouest,HT,11,Haiti,3722286.0,Léogâne,134190.0
17130,MG.12,Vakinankaratra,MG,12,Madagascar,1065140.0,Faratsiho,37563.0


Можно посмотреть распределение популяции людей по континентам. Для этого нам понадобится датасет с данными, какая стран относится к какому континенту. Создадим отдельную таблицу со странами, континентами и их популяцией.

In [15]:
continents = pd.read_csv('country-and-continent-codes-list-csv.csv', delimiter=',')
continents.info()
continents.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 262 entries, 0 to 261
Data columns (total 6 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Continent_Name             262 non-null    object 
 1   Continent_Code             219 non-null    object 
 2   Country_Name               262 non-null    object 
 3   Two_Letter_Country_Code    261 non-null    object 
 4   Three_Letter_Country_Code  258 non-null    object 
 5   Country_Number             258 non-null    float64
dtypes: float64(1), object(5)
memory usage: 12.4+ KB


,Continent_Name,Continent_Code,Country_Name,Two_Letter_Country_Code,Three_Letter_Country_Code,Country_Number
0,Asia,AS,"Afghanistan, Islamic Republic of",AF,AFG,4.0
1,Europe,EU,"Albania, Republic of",AL,ALB,8.0
2,Antarctica,AN,Antarctica (the territory South of 60 deg S),AQ,ATA,10.0
3,Africa,AF,"Algeria, People's Democratic Republic of",DZ,DZA,12.0
4,Oceania,OC,American Samoa,AS,ASM,16.0


In [16]:
y = joined_df.groupby(['country_code'])['population'].sum()
continent_country = continents.join(y, on='Two_Letter_Country_Code',)
continent_country = continent_country.dropna(subset=['population'])
continent_country.info()
continent_country.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 231 entries, 0 to 257
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Continent_Name             231 non-null    object 
 1   Continent_Code             197 non-null    object 
 2   Country_Name               231 non-null    object 
 3   Two_Letter_Country_Code    231 non-null    object 
 4   Three_Letter_Country_Code  231 non-null    object 
 5   Country_Number             231 non-null    float64
 6   population                 231 non-null    float64
dtypes: float64(2), object(5)
memory usage: 14.4+ KB


,Continent_Name,Continent_Code,Country_Name,Two_Letter_Country_Code,Three_Letter_Country_Code,Country_Number,population
0,Asia,AS,"Afghanistan, Islamic Republic of",AF,AFG,4.0,8515076.0
1,Europe,EU,"Albania, Republic of",AL,ALB,8.0,1349299.0
3,Africa,AF,"Algeria, People's Democratic Republic of",DZ,DZA,12.0,20057133.0
4,Oceania,OC,American Samoa,AS,ASM,16.0,11500.0
5,Europe,EU,"Andorra, Principality of",AD,AND,20.0,36283.0


In [17]:
cont = continent_country.groupby(['Continent_Name'])['population'].sum()
cont = pd.DataFrame(cont)
cont

,population
Continent_Name,
Africa,4.075750e+08
Antarctica,4.500000e+01
Asia,1.835620e+09
Europe,5.841259e+08
North America,3.746286e+08
Oceania,3.201189e+07
South America,2.797527e+08


Получили примерные цифры популиции населения имеющихся у нас в выборке городов.

### Отсеивание стран

По заданию, нам необходимы только такие страны, как Россия, Беларусь, Армения, Казахстан, Кыргызстан, Турция, Сербия. Поэтому ограничим данные в датасете только ими.

In [20]:
COUNTRY_CODES

['RU', 'BY', 'KG', 'KZ', 'AM', 'TR', 'RS']

In [21]:
selected_cities = joined_df[joined_df['country_code'].isin(COUNTRY_CODES)].reset_index(drop=True)

## Обучение моделей

Для проведения тестов у нас имеется набор данных, каторые содержат запрос *query* и ожидаемый результат запроса.

По заданию данные необходимо хранить в PostgreSQL, следовательно удем используем SQLAlchemy для работы с базой.

Создадим объект для подключения к базе.

In [23]:
engine = create_engine(URL(**DATABASE))

### Подготовка данных.

Загрузим датасет с тестовыми данными.

In [25]:
true_set = pd.read_csv('geo_test.csv', delimiter=';')
true_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348 entries, 0 to 347
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   query    348 non-null    object
 1   name     348 non-null    object
 2   region   348 non-null    object
 3   country  348 non-null    object
dtypes: object(4)
memory usage: 11.0+ KB


Теперь необходимо разделить датасет на features И target.

In [26]:
X_true = true_set['query']
y_true = true_set.drop('query', axis=1)

### Обучение моделей

LaBSE (language-agnostic BERT sentence embeddings) – это модель, основная задача которой сближать друг с другом эмбеддинги предложений с одинаковым смыслом на разных языках. Благодаря этой способности можно, например, обучать модель классифицировать английские тексты, а потом применять на русских, или находить в большом корпусе пары предложений на разных языках, являющиеся переводами друг друга.

Для обучения есть 4 модели на основе LaBSE с разным количсеством эпох (1, 3, 5, 7), которые загружются для проверки через SentenceTransformer.

In [58]:
model_names = ['LaBSE-geonames-1e','LaBSE-geonames-3e','LaBSE-geonames-5e','LaBSE-geonames-7e']
results_df = pd.DataFrame(columns=['Model', 'Accuracy'])
inspector = inspect(engine)

for model_name in model_names:
    EMBEDDER = SentenceTransformer(model_names)
    epochs = model_name.split('-')[-1]
    table_name = f'corpus_embeddings_{epochs}'
    
    if inspector.has_table(table_name):
        query = f'SELECT * FROM {table_name}'
        CORPUS_EMBEDDINGS = pd.read_sql_query(query,con=engine).astype('float32').values
    else:
        corpus = selected_cities['name_city']
        CORPUS_EMBEDDINGS = EMBEDDER.encode(corpus, convert_to_tensor=False)
        pd.DataFrame(CORPUS_EMBEDDINGS).to_sql( table_name, con=engine,if_exists='replace',index=False)

    y_inference = predict_val(X_true)  # Функция предсказания
    accuracy = accuracy_score(y_true, y_inference)  # Функция оценки точности

    results_df = pd.concat([results_df, pd.DataFrame({'Model': [model_name], 'Accuracy': [accuracy]})], ignore_index=True)

Выведем результаты оценки моделей.

In [31]:
results_df = results_df.sort_values(by='Accuracy', ascending=False)
results_df

,Model,Accuracy
0,LaBSE-geonames-1e,0.8600
1,LaBSE-geonames-3e,0.8333
2,LaBSE-geonames-5e,0.7767
3,LaBSE-geonames-7e,0.7767


Как можно увидеть, лучшей себя показала модель обученная на 1 эпохе и результатом *accuracy* 0.86.

Произведем загрузку лучшей модели.

In [35]:
best_model_name = results_df.iloc[0]['Model']
EMBEDDER = SentenceTransformer(best_model_name)
epochs = best_model_name.split('-')[-1]
best_table_name = f'corpus_embeddings_{epochs}'
query = f'SELECT * FROM {best_table_name}'
CORPUS_EMBEDDINGS = pd.read_sql_query(query,con=engine).astype('float32').values

Сохраним данные датафреймов в БД.

In [36]:
cities15000.to_sql(
    'cities15000',
    con=engine,
    if_exists='replace',
    index=False
)
countries.to_sql(
    'countries', 
    con=engine,
    if_exists='replace',
    index=False
)
admin_codes.to_sql(
    'admin_codes',
    con=engine,
    if_exists='replace',
    index=False
)
selected_cities.to_sql(
    'selected_cities', 
    con=engine, 
    if_exists='replace',
    index=False
)

713

## Проверка модели

Для проверки работоспособности модели возьмем искаженное название `Мгнитогрс` и вывведем данные.

По заданию результат должен возвращаться как список словарей.

In [39]:
similar('Мгнитогрс', top_k=5, is_dictionary=True)

[{'name': 'Magnitogorsk',
  'code': 'RU.13',
  'region': 'Chelyabinsk',
  'country': 'Russia',
  'similarity': 0.5652748346328735},
 {'name': 'Mednogorsk',
  'code': 'RU.55',
  'region': 'Orenburg Oblast',
  'country': 'Russia',
  'similarity': 0.4003976285457611},
 {'name': 'Dimitrovgrad',
  'code': 'RU.81',
  'region': 'Ulyanovsk',
  'country': 'Russia',
  'similarity': 0.3825973868370056},
 {'name': "Mezgor'e",
  'code': 'RU.08',
  'region': 'Bashkortostan Republic',
  'country': 'Russia',
  'similarity': 0.36654436588287354},
 {'name': 'Manturovo',
  'code': 'RU.37',
  'region': 'Kostroma Oblast',
  'country': 'Russia',
  'similarity': 0.36367714405059814}]

Модель определила наиболее подходящие горрода `Магнитогорск` и `Медногорск`.

## Вывод

Итак, имелась задачать построить модель для подбора наиболее подходящих унифицированных названий городов.

1. Была проведена загрузка и исследвание данных с сайта `Geonames.org`

Выборка состоит из следующих файлов:
- admin1CodesASCII - административные коды городов
- cities15000 - названия городов и их данные
- countryInfo - коды стран

На их основе получен общий датасет, в котором произведена выборка для России, Беларуси, Армении, Казахстана, Кыргызстана, Турции, Сербии.


2. Для обучение выбрана модель LaBSE и обучена на разном количестве эпох (1, 3, 5, 7), которые загружются для проверки через SentenceTransformer.

По результатам обучение получены следующие метрики:
*   LaBSE-geonames-1e	`0.8600`
*	LaBSE-geonames-3e	`0.8333`
*	LaBSE-geonames-5e	`0.7767`
*	LaBSE-geonames-7e	`0.7767`

Как видим лучшей оказалась модель, обученная на 1 эпохе.

3. Проверка модели показала, что она достаточно хорошо определяет подходящие города.

В качестве примера взято искаженное название `Мгнитогрс` и получены наиболее подходящие `Магнитогорск` и `Медногорск`.